# Import modules

In [1]:
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Alphabet import IUPAC
from pathlib import Path
from typing import Union
import pandas as pd

try:
    import import_notebook
except ModuleNotFoundError:
    pass

import code_synthesis

importing Jupyter notebook from code_synthesis.ipynb


# Data Process Functions

# Generate NNS primers

In [2]:
def test_generate_primers(): 
    dna = 'GGCAATGCGcccccaATGaaaaaaTAAaaACGGGGTTTTaaa'
    start = ('GGCAATGCGccccca')
    end = ('aaACGGGGTTTTaaa')
    
    df_prim = code_synthesis.generate_primers(dna, start, end, output_file=None,
                           codon='NNS', length_primer=18, tm=60, return_df=True)
    assert len(df_prim)== 9, 'error the primer length does not match'
    assert df_prim.iloc[1, 1] == 'NNSGCGCCCCCAATGAAAAAATAAAAACGGG', 'error the first primer is not mutated at Met'
    assert df_prim.iloc[8, 1] == 'CGCCCCCAATGAAAAAANNSAAACGGGGTTTTAA', 'error the last primer is not mutated at the stop codon'


In [3]:
test_generate_primers()

In [83]:
def test_primerdesign(): 
    dna = 'GGCAATGCGcccccaATGaaaaaaTAAaaACGGGGTTTTaaa' #42 codons 
    primers0 = code_synthesis._primerdesign(dna, codon = 'NNS', codonposition = 0, 
                                            length_primer=None, tm = 60)
    primers1 = code_synthesis._primerdesign(dna, codon = 'NNS', codonposition = 1, 
                                            length_primer=None, tm = 60) 
    primers2 = code_synthesis._primerdesign(dna, codon = 'NNS', codonposition = 2, 
                                            length_primer=None, tm = 60)  
    primers3 = code_synthesis._primerdesign(dna, codon = 'NNS', codonposition = 3, 
                                            length_primer=None, tm = 60)
    primers21 = code_synthesis._primerdesign(dna, codon = 'NNS', codonposition = 21, 
                                           length_primer = 18, tm = None)
    assert type(primers3) == tuple, 'error the output is not a tuple'
    
    assert primers0 == ('NNSAATGCGcccccaATGaaaaaaTAAaaACGG', 'CCGTttTTAttttttCATtgggggCGCATTSNN'), 'error the output is not the forward and reverse primers at codon position 0'
    assert primers1 == ('NNSATGCGcccccaATGaaaaaaTAAaaACGGG', 'CCCGTttTTAttttttCATtgggggCGCATSNN'), 'error the output is not the forward and reverse primers at codon position 1'
    assert primers2 == ('NNSTGCGcccccaATGaaaaaaTAAaaACGGG', 'CCCGTttTTAttttttCATtgggggCGCASNN'), 'error the output is not the forward and reverse primers at codon position 2'
    assert primers3 == ('NNSGCGcccccaATGaaaaaaTAAaaACGGG', 'CCCGTttTTAttttttCATtgggggCGCSNN'), 'error the output is not the forward and reverse primers at codon position 3'

    assert primers21 == ('AATGCGcccccaATGaaaNNSTAAaaACGGGGTTTTaaa', 'tttAAAACCCCGTttTTASNNtttCATtgggggCGCATT'), 'error the codon is not replaced after position 21'


In [84]:
test_primerdesign()

In [107]:
def test_create_primers_list(): 
    dna = 'GGCAATGCGcccccaATGaaaaaaTAAaaACGGGGTTTTaaa'

    primerslist = code_synthesis._create_primers_list(dna, start_codon = 3, end_codon = 6, 
                                                      codon = 'NNS', length_primer = 15, 
                                                      tm = 60)
    assert type(primerslist[0]) == list, 'error the type is not a list' 

In [108]:
test_create_primers_list()